In [1]:
import pandas as pd
import numpy as np

In [2]:
maids, email, phone = pd.read_csv("maids.csv"), pd.read_csv("phone.csv"), pd.read_csv("email.csv")

In [11]:
phone.columns

Index(['vb_email', 'vb_tsmart_first_name', 'vb_tsmart_middle_name',
       'vb_tsmart_last_name', 'vb_voterbase_phone',
       'vb_voterbase_phone_wireless', 'vb_voterbase_dob',
       'coalesced_noncommercial_age', 'gender_noncommercial',
       'vb_tsmart_exact_track', 'vb_tsmart_full_address', 'vb_tsmart_city',
       'vb_tsmart_state', 'vb_tsmart_zip', 'vb_tsmart_zip4',
       'vb_tsmart_street_number', 'vb_tsmart_street_name',
       'vb_tsmart_street_suffix', 'vb_vf_reg_cass_street_suffix',
       'ts_reg_urbanicity_bins', 'throtle_id', 'md5_lower', 'sha1_lower',
       'sha256_lower', 'id'],
      dtype='object')

In [3]:
f1, f2, f3, f4 = pd.read_csv("final_table/f1_both_email_phone.csv"),\
                pd.read_csv("final_table/f2_phone_only.csv"), \
                pd.read_csv("final_table/f3_email_only.csv"), \
               pd.read_csv("final_table/f4_both_missing.csv")

In [4]:
f1.shape[0] + f2.shape[0] + f3.shape[0] + f4.shape[0] 

1000000

In [5]:
for df in [f1, f2, f3, f4]:
    df.fillna('-', inplace=True)
    df['id'] = df.fn + df.ln + df.dob.astype(str) + df.st + df.zip.astype(str)

In [6]:
for df in [maids, email, phone]:
    df.fillna('-', inplace=True)
    df['id'] = df.vb_tsmart_first_name + df.vb_tsmart_last_name + df.vb_voterbase_dob.astype(str) + \
    df.vb_tsmart_state + df.vb_tsmart_zip.astype(str)

In [15]:
dfs = [f1, f2, f3, f4]
for i in range(0, len(dfs)):
    df = dfs[i]
    # first create phone sub
    phone_sub = df.merge(phone, on='id', how='left')
    cols = ['Unnamed: 0', 'fn', 'ln', 'vb_voterbase_phone', 'vb_voterbase_phone_wireless', 
           'dob', 'age', 'gen', 'ct', 'st', 'zip', 'vb_email', 'country', 'id']

    # get all records from phone table after merge
    # rename for appending
    # this is all the new phone, email from phone dataset
    phone_sub_app = phone_sub[cols].rename(columns={'vb_voterbase_phone' : 'phone1',
                                                    'vb_voterbase_phone_wireless' : 'phone2',
                                                    'vb_email' : 'email',
                                                   })
    
    # second create maid sub
    maid_sub = df.merge(maids, on='id', how='left')
    cols = ['Unnamed: 0', 'fn', 'ln', 'vb_voterbase_phone', 'vb_voterbase_phone_wireless',
            'dob', 'age', 'gen', 'ct', 'st', 'zip', 'vb_email', 'country', 'id']
    # get all records from maid table after merge
    # rename for appending
    # this is all the new phone, email from maid dataset
    maid_sub_app = maid_sub[cols].rename(columns={'vb_voterbase_phone' : 'phone1',
                                                  'vb_voterbase_phone_wireless' : 'phone2',
                                                  'vb_email' : 'email'})
    
    
    # appending
    new_cols = list(df.columns) 
    maid_subf, phone_subf = maid_sub_app[new_cols], phone_sub_app[new_cols]
    df_enriched = df.append(maid_subf)
    df_enriched = df_enriched.append(phone_subf)
    cols = list(df_enriched.columns) + ['native_maid', 'throtle_id']
    # add maid value to df 
    df_enriched_maid = df_enriched.merge(maids, on='id', how='left')[cols]
    df_enriched_maid = df_enriched_maid.rename(columns={'native_maid' : 'madid'})
    df_enriched_maid_dedup = df_enriched_maid.drop_duplicates()
    print("Shape after de-duplicates for: " + str(i + 1), df_enriched_maid_dedup.shape[0])
    df_enriched_maid_dedup = df_enriched_maid_dedup[df_enriched_maid_dedup.columns[1:]]
    # fix phone number, dob and gender issue
    
    df_enriched_maid_dedup.to_csv("final_table/f" + str(i + 1) + '_enriched.csv')

Index(['Unnamed: 0', 'fn', 'ln', 'phone1', 'phone2', 'dob', 'age', 'gen', 'ct',
       'st', 'zip', 'email', 'country', 'id'],
      dtype='object')
Shape after de-duplicates for: 1 1163580
Index(['Unnamed: 0', 'fn', 'ln', 'phone1', 'phone2', 'dob', 'age', 'gen', 'ct',
       'st', 'zip', 'email', 'country', 'id'],
      dtype='object')
Shape after de-duplicates for: 2 934000
Index(['Unnamed: 0', 'fn', 'ln', 'phone1', 'phone2', 'dob', 'age', 'gen', 'ct',
       'st', 'zip', 'email', 'country', 'id'],
      dtype='object')
Shape after de-duplicates for: 3 105164
Index(['Unnamed: 0', 'fn', 'ln', 'phone1', 'phone2', 'dob', 'age', 'gen', 'ct',
       'st', 'zip', 'email', 'country', 'id'],
      dtype='object')
Shape after de-duplicates for: 4 380116


In [16]:
maids[]

0          4.18781e+07
1          4.18781e+07
2                    -
3                    -
4                    -
              ...     
1534016    1.98751e+08
1534017              -
1534018              -
1534019    2.05905e+07
1534020    2.05905e+07
Name: throtle_id, Length: 1534021, dtype: object